# Timing Tests of `fastnumbers` Functions Compared to Equivalent Solutions

In order for you to see the benefit of `fastnumbers`, some timings are collected below for comparison to equivalent python implementations. The numbers may change depending on the machine you are on or the Python version you are using.

Feel free to download this Jupyter Notebook and run the tests yourself to see how `fastnumbers` performs on your machine (it takes about 1-2 minutes total).

#### Some notes about the data

 - Each test is the time it takes for the function to run 100,000 times on a given input.
 - Each test is repeated either 5 or 100 times, and the mean ± standard deviation is reported.
 - The fastest time is shown in **bold**
 - The timing results for the pure-Python functions include about 10-15 ms of "function call overhead"; the `fastnumbers` functions do not suffer from as much overhead because they are C-extensions.
 - Python version-dependent behaviors:
   - **Python 2.7** has a particularly slow `int` function, so the `fastnumbers` speedup is much larger on Python 2.7 than Python 3.x
   - **Python >=3.6** is slightly slower in general than previous versions because underscores are now allowed in floats and integers which makes parsing take a bit longer due to the extra logic.
   
#### Notes about the `Timing` class below

The timing runner class is implemented below, and this is used in all the tests to perform the actual timing tests in the sections below. In general you can skip this implementation, but of note is the `THINGS_TO_TIME` tuple, which contains the values that are passed to the functions to type the various input types.

In [1]:
from __future__ import print_function, division
import re
import math
import timeit
from IPython.display import Markdown, display, clear_output

class Timer(object):
    """Class to time functions and make pretty tables of the output."""
    
    # This is a list of all the things we will time with an associated label.
    THINGS_TO_TIME = (
        ('not_a_number', 'Non-number String'),
        ('-41053', 'Int String'),
        ('35892482945872302493947939485729', 'Large Int String'),
        ('-41053.543034e34', 'Float String'),
        ('-41053.543028758302e256', 'Large Float String'),
        (-41053, 'Int'),
        (-41053.543028758302e100, 'Float'),
    )

    # Formatting strings.
    FUNCTION_CALL_FMT = '{}({!r})'
    
    def __init__(self, title):
        display(Markdown('### ' + title))
        self.functions = []
    
    def add_function(self, func, label, setup='pass'):
        """Add a function to be timed and compared."""
        self.functions.append((func, setup, label))

    def time_functions(self, repeat=5):
        """Time all the given functions against all input then display results."""

        # Collect the function labels to make the header of this table.
        # Show that the units are seconds for each.
        function_labels = [label + ' (ms)' for _, _, label in self.functions]
        
        # Construct the table strings, formatted in Markdown.
        # Store each line as a string element in a list.
        # This portion here is the table header only for now.
        table = Table()
        table.add_header('Input type', *function_labels)
        
        # For each value, time each function and collect the results.
        for value, value_label in self.THINGS_TO_TIME:
            row = []
            for func, setup, _ in self.functions:
                call = self.FUNCTION_CALL_FMT.format(func, value)
                try:
                    row.append(self._timeit(call, setup, repeat))
                except (ValueError, TypeError):
                    # We might send in some invalid input accidentally.
                    # Ignore those inputs.
                    break

            # Only add this row if the for loop quit without break.
            else:
                # Convert to milliseconds
                row = [(mean * 1000, stddev * 1000) for mean, stddev in row]
                # Make the lowest value bold.
                min_indx = min(enumerate(row), key=lambda x: x[1])[0]
                row = ['{:.3f} ± {:.3f}'.format(*x) for x in row]
                row[min_indx] = self.bold(row[min_indx])
                table.add_row(value_label, *row)

        # Show the results in a table.
        display(Markdown(str(table)))

    @staticmethod
    def mean(x):
        return math.fsum(x) / len(x)

    @staticmethod
    def stddev(x):
        mean = Timer.mean(x)
        sum_of_squares = math.fsum((v - mean)**2 for v in x)
        return math.sqrt(sum_of_squares / (len(x) - 1))

    @staticmethod
    def bold(x):
        return "**{}**".format(x)
    
    def _timeit(self, call, setup, repeat=5):
        """Perform the actual timing and return a formatted string of the runtime"""
        result = timeit.repeat(call, setup, number=100000, repeat=repeat)
        return self.mean(result), self.stddev(result)

class Table(list):
    """List of strings that can be made into a Markdown table."""
    def add_row(self, *elements):
        self.append('|'.join(elements))
    def add_header(self, *elements):
        self.add_row(*elements)
        seperators = ['---'] * len(elements)
        seperators = [sep + (':' if i != 0 else '') for i, sep in enumerate(seperators)]
        self.add_row(*seperators)
    def __str__(self):
        return '\n'.join(self)


## Built-in Functions Drop-in Replacement Timing Results
The following timing tests compare the performance of Python's builtin `int` and `float` functions against the implementations from `fastnumbers` for various input types.

In [2]:
timer = Timer('Timing comparison of `int` functions')
timer.add_function('int', 'builtin')
timer.add_function('int', 'fastnumbers', 'from fastnumbers import int')
timer.time_functions(repeat=100)

### Timing comparison of `int` functions

Input type|builtin (ms)|fastnumbers (ms)
---|---:|---:
Int String|35.323 ± 9.081|**23.572 ± 7.440**
Large Int String|40.238 ± 10.446|**34.712 ± 2.476**
Int|15.358 ± 1.650|**14.838 ± 1.456**
Float|32.926 ± 2.585|**30.515 ± 2.103**

In [3]:
timer = Timer('Timing comparison of `float` functions')
timer.add_function('float', 'builtin')
timer.add_function('float', 'fastnumbers', 'from fastnumbers import float')
timer.time_functions(repeat=100)

### Timing comparison of `float` functions

Input type|builtin (ms)|fastnumbers (ms)
---|---:|---:
Int String|22.858 ± 2.604|**19.742 ± 1.307**
Large Int String|**57.890 ± 3.036**|77.814 ± 18.424
Float String|52.215 ± 5.904|**25.298 ± 5.004**
Large Float String|**98.275 ± 14.311**|105.949 ± 25.405
Int|22.629 ± 4.934|**19.964 ± 4.926**
Float|17.387 ± 3.155|**15.700 ± 4.025**

## Error-Handling Conversion Functions Timing Results
The following timing tests compare the performance of the `fastnumbers` functions that convert input to numeric types while doing error handling with common equivalent pure-Python implementations.

In [4]:
def int_re(x, int_match=re.compile(r'[-+]?\d+$').match):
    """Function to simulate fast_int but with regular expressions."""
    try:
        if int_match(x):
            return int(x)
        else:
            return x
    except TypeError:
        return int(x)

def int_try(x):
    """Function to simulate fast_int but with try/except."""
    try:
        return int(x)
    except ValueError:
        return x

timer = Timer('Timing comparison of `int` functions with error handling')
timer.add_function('int_try', 'try/except', 'from __main__ import int_try')
timer.add_function('int_re', 'regex', 'from __main__ import int_re')
timer.add_function('fast_int', 'fastnumbers', 'from fastnumbers import fast_int')
timer.time_functions()

### Timing comparison of `int` functions with error handling

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|288.140 ± 15.103|95.849 ± 22.142|**29.397 ± 7.606**
Int String|45.669 ± 7.282|128.812 ± 6.779|**23.549 ± 2.488**
Large Int String|46.389 ± 2.725|164.365 ± 29.678|**35.587 ± 4.624**
Float String|304.316 ± 39.024|89.097 ± 12.753|**21.455 ± 1.205**
Large Float String|304.527 ± 19.778|82.127 ± 2.138|**20.174 ± 2.920**
Int|30.321 ± 2.449|193.256 ± 53.063|**17.865 ± 1.575**
Float|53.678 ± 4.227|252.511 ± 59.851|**31.799 ± 2.202**

In [5]:
def float_re(x, float_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate fast_float but with regular expressions."""
    try:
        if float_match(x):
            return float(x)
        else:
            return x
    except TypeError:
        return float(x)

def float_try(x):
    """Function to simulate fast_float but with try/except."""
    try:
        return float(x)
    except ValueError:
        return x

timer = Timer('Timing comparison of `float` functions with error handling')
timer.add_function('float_try', 'try/except', 'from __main__ import float_try')
timer.add_function('float_re', 'regex', 'from __main__ import float_re')
timer.add_function('fast_float', 'fastnumbers', 'from fastnumbers import fast_float')
timer.time_functions()

### Timing comparison of `float` functions with error handling

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|188.898 ± 18.676|73.262 ± 8.006|**18.884 ± 2.835**
Int String|38.404 ± 2.995|178.078 ± 48.635|**21.528 ± 2.384**
Large Int String|76.017 ± 4.717|254.543 ± 15.861|**70.833 ± 2.167**
Float String|106.071 ± 23.522|263.581 ± 49.543|**26.335 ± 1.996**
Large Float String|216.434 ± 28.086|390.196 ± 110.809|**162.933 ± 41.173**
Int|37.703 ± 4.835|236.041 ± 53.893|**29.308 ± 4.156**
Float|31.519 ± 3.201|183.121 ± 14.039|**14.708 ± 0.228**

In [6]:
def real_re(x,
            int_match=re.compile(r'[-+]?\d+$').match,
            real_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate fast_real but with regular expressions."""
    try:
        if int_match(x):
            return int(x)
        elif real_match(x):
            return float(x)
        else:
            return x
    except TypeError:
        if type(x) in (float, int):
            return x
        else:
            raise TypeError

def real_try(x):
    """Function to simulate fast_real but with try/except."""
    try:
        a = float(x)
    except ValueError:
        return x
    else:
        b = int(a)
        return b if a == b else b

timer = Timer('Timing comparison of `float` (but coerce to `int` if possible) functions with error handling')
timer.add_function('real_try', 'try/except', 'from __main__ import real_try')
timer.add_function('real_re', 'regex', 'from __main__ import real_re')
timer.add_function('fast_real', 'fastnumbers', 'from fastnumbers import fast_real')
timer.time_functions()

### Timing comparison of `float` (but coerce to `int` if possible) functions with error handling

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|207.852 ± 56.663|149.943 ± 45.688|**18.546 ± 2.607**
Int String|89.592 ± 19.490|127.236 ± 22.242|**25.066 ± 3.208**
Large Int String|179.318 ± 62.781|199.194 ± 47.163|**45.585 ± 3.892**
Float String|174.512 ± 32.356|302.395 ± 13.201|**52.468 ± 15.752**
Large Float String|377.553 ± 103.149|484.915 ± 138.740|**168.186 ± 23.018**
Int|78.407 ± 16.058|178.766 ± 8.596|**15.973 ± 2.283**
Float|105.982 ± 3.780|202.229 ± 17.673|**83.183 ± 5.225**

In [7]:
def forceint_re(x,
                int_match=re.compile(r'[-+]\d+$').match,
                float_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate fast_forceint but with regular expressions."""
    try:
        if int_match(x):
            return int(x)
        elif float_match(x):
            return int(float(x))
        else:
            return x
    except TypeError:
        return int(x)

def forceint_try(x):
    """Function to simulate fast_forceint but with try/except."""
    try:
        return int(x)
    except ValueError:
        try:
            return int(float(x))
        except ValueError:
            return x

timer = Timer('Timing comparison of forced `int` functions with error handling')
timer.add_function('forceint_try', 'try/except', 'from __main__ import forceint_try')
timer.add_function('forceint_re', 'regex', 'from __main__ import forceint_re')
timer.add_function('fast_forceint', 'fastnumbers', 'from fastnumbers import fast_forceint')
timer.time_functions()

### Timing comparison of forced `int` functions with error handling

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|623.751 ± 99.635|226.865 ± 101.427|**33.186 ± 1.592**
Int String|194.545 ± 64.157|319.450 ± 158.132|**69.576 ± 25.294**
Large Int String|**64.022 ± 2.892**|782.087 ± 171.279|113.056 ± 71.338
Float String|1485.072 ± 297.423|456.711 ± 113.161|**53.187 ± 7.833**
Large Float String|593.409 ± 80.863|550.320 ± 135.047|**206.773 ± 37.612**
Int|57.952 ± 4.483|224.015 ± 35.008|**29.727 ± 11.793**
Float|**68.722 ± 7.373**|286.956 ± 49.755|103.735 ± 28.160

In [ ]:
## Checking Functions Timing Results
The following timing tests compare the performance of the `fastnumbers` functions that check if an input *could* be converted to numeric type with common equivalent pure-Python implementations.

In [8]:
def isint_re(x, int_match=re.compile(r'[-+]?\d+$').match):
    """Function to simulate isint but with regular expressions."""
    t = type(x)
    return t == int if t in (float, int) else bool(int_match(x))

def isint_try(x):
    """Function to simulate isint but with try/except."""
    try:
        int(x)
    except ValueError:
        return False
    else:
        return type(x) != float

timer = Timer('Timing comparison to check if value can be converted to `int`')
timer.add_function('isint_try', 'try/except', 'from __main__ import isint_try')
timer.add_function('isint_re', 'regex', 'from __main__ import isint_re')
timer.add_function('isint', 'fastnumbers', 'from fastnumbers import isint')
timer.time_functions()

### Timing comparison to check if value can be converted to `int`

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|1085.031 ± 254.837|522.500 ± 76.186|**69.979 ± 45.260**
Int String|219.463 ± 63.626|333.195 ± 112.811|**20.506 ± 2.018**
Large Int String|97.451 ± 32.370|218.346 ± 56.408|**31.865 ± 2.791**
Float String|346.459 ± 36.978|152.887 ± 19.076|**20.490 ± 0.426**
Large Float String|322.876 ± 42.671|149.253 ± 17.148|**23.705 ± 3.178**
Int|71.525 ± 18.842|52.629 ± 11.202|**21.195 ± 1.710**
Float|74.726 ± 5.848|47.925 ± 1.233|**16.673 ± 2.041**

In [9]:
def isfloat_re(x, float_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate isfloat but with regular expressions."""
    t = type(x)
    return t == float if t in (float, int) else bool(float_match(x))

def isfloat_try(x):
    """Function to simulate isfloat but with try/except."""
    try:
        float(x)
    except ValueError:
        return False
    else:
        return type(x) != int

timer = Timer('Timing comparison to check if value can be converted to `float`')
timer.add_function('isfloat_try', 'try/except', 'from __main__ import isfloat_try')
timer.add_function('isfloat_re', 'regex', 'from __main__ import isfloat_re')
timer.add_function('isfloat', 'fastnumbers', 'from fastnumbers import isfloat')
timer.time_functions()

### Timing comparison to check if value can be converted to `float`

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|182.210 ± 20.122|133.694 ± 20.650|**25.168 ± 7.070**
Int String|64.257 ± 9.972|164.439 ± 7.438|**20.164 ± 2.501**
Large Int String|136.765 ± 36.040|241.040 ± 63.798|**22.109 ± 0.541**
Float String|141.777 ± 30.698|202.393 ± 34.116|**23.061 ± 1.878**
Large Float String|171.151 ± 25.088|229.452 ± 30.970|**22.998 ± 1.441**
Int|69.957 ± 13.344|57.172 ± 3.851|**19.001 ± 3.021**
Float|66.538 ± 11.738|45.700 ± 2.240|**16.773 ± 2.202**

In [10]:
def isreal_re(x, real_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate isreal but with regular expressions."""
    return type(x) in (float, int) or bool(real_match(x))

def isreal_try(x):
    """Function to simulate isreal but with try/except."""
    try:
        float(x)
    except ValueError:
        return False
    else:
        return True

timer = Timer('Timing comparison to check if value can be converted to `float` or `int`')
timer.add_function('isreal_try', 'try/except', 'from __main__ import isreal_try')
timer.add_function('isreal_re', 'regex', 'from __main__ import isreal_re')
timer.add_function('isreal', 'fastnumbers', 'from fastnumbers import isreal')
timer.time_functions()

### Timing comparison to check if value can be converted to `float` or `int`

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|174.858 ± 14.229|138.049 ± 20.184|**18.950 ± 2.466**
Int String|47.278 ± 17.332|241.620 ± 28.273|**35.022 ± 11.416**
Large Int String|99.399 ± 32.419|206.508 ± 13.849|**23.211 ± 1.486**
Float String|93.225 ± 21.947|248.304 ± 51.405|**37.939 ± 12.305**
Large Float String|135.025 ± 21.534|206.936 ± 20.956|**38.715 ± 7.300**
Int|38.894 ± 3.122|42.918 ± 5.559|**13.829 ± 0.503**
Float|30.978 ± 4.206|38.609 ± 2.707|**14.455 ± 0.931**

In [11]:
def isintlike_re(x,
                 int_match=re.compile(r'[-+]?\d+$').match,
                 float_match=re.compile(r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?$').match):
    """Function to simulate isintlike but with regular expressions."""
    try:
        if int_match(x):
            return True
        elif float_match(x):
            return float(x).is_integer()
        else:
            return False
    except TypeError:
        return int(x) == x

def isintlike_try(x):
    """Function to simulate isintlike but with try/except."""
    try:
        a = int(x)
    except ValueError:
        try:
            a = float(x)
        except ValueError:
            return False
        else:
            return a.is_integer()
    else:
        return a == float(x)

timer = Timer('Timing comparison to check if value can be coerced losslessly to `int`')
timer.add_function('isintlike_try', 'try/except', 'from __main__ import isintlike_try')
timer.add_function('isintlike_re', 'regex', 'from __main__ import isintlike_re')
timer.add_function('isintlike', 'fastnumbers', 'from fastnumbers import isintlike')
timer.time_functions()

### Timing comparison to check if value can be coerced losslessly to `int`

Input type|try/except (ms)|regex (ms)|fastnumbers (ms)
---|---:|---:|---:
Non-number String|507.209 ± 57.752|136.108 ± 32.232|**18.035 ± 0.499**
Int String|90.153 ± 4.045|76.587 ± 2.458|**17.534 ± 1.329**
Large Int String|153.295 ± 5.119|115.146 ± 4.310|**21.307 ± 0.250**
Float String|464.332 ± 34.784|437.892 ± 115.270|**21.303 ± 2.797**
Large Float String|671.698 ± 95.556|566.724 ± 150.177|**43.428 ± 6.251**
Int|83.071 ± 15.900|195.620 ± 11.114|**15.590 ± 2.398**
Float|120.014 ± 21.239|275.652 ± 34.122|**59.772 ± 10.166**